# CSD y CM de Actividad Evocada. 
Usemos esto como opurtunidad para poner todo el código en orden.


En este *notebook* de Julia implementaremos métodos numéricos para llevar a cabo un análisis de *centro de masa* (promedio vectorial) de la densidad de pozos y fuentes de corriente sobre una rebanada del hipocámpo de una rata. En este Notebook estudiaremos los datos de los experimentos evocados que hizo Franco en Agosto.
 Al parecer primero toca hacer una prueba con todo aquí mismo antes que hacer un modulo ordenado por una inconveniencia al hacer modulos (no se pueden recargar).

In [4]:
using HDF5

In [5]:
function AbreyCheca(x::AbstractString)
    #Abre el archivo de brw (acepta el nombre tal cual)
    archivo=h5open(x)
    #sacatito todas las variables que te interesan
    numcuadros=archivo["/3BRecInfo/3BRecVars/NRecFrames"][1][1]
    frecuencia=archivo["/3BRecInfo/3BRecVars/SamplingRate"][1][1]
    maxvolt=archivo["/3BRecInfo/3BRecVars/MaxVolt"][1][1]
    minvolt=archivo["/3BRecInfo/3BRecVars/MinVolt"][1][1]
    bitdepth=archivo["/3BRecInfo/3BRecVars/BitDepth"][1][1]
    duracionexperimento=numcuadros/frecuencia
    factordeescala=(maxvolt-minvolt)/2^bitdepth
    DatosCrudos=read(archivo["/3BData/Raw"])
    result=Dict("numcuadros" => numcuadros,
                "frecuencia"=> frecuencia,
                "maxvolt" => maxvolt,
                "minvolt" => minvolt,
                "bitdepth" => bitdepth,
                "duracion" => duracionexperimento,
                "factor " => factordeescala,
                "DatosCrudos"=>DatosCrudos )
    return result
                    
end

AbreyCheca (generic function with 1 method)

In [6]:
Datos=AbreyCheca("050815_1R4.brw")

Dict{ASCIIString,Any} with 8 entries:
  "minvolt"     => -4125.0
  "maxvolt"     => 4125.0
  "factor "     => 2.01416015625
  "frecuencia"  => 7020.800706028248
  "DatosCrudos" => 4096x210624 Array{UInt16,2}:…
  "bitdepth"    => 0x0c
  "numcuadros"  => 210624
  "duracion"    => 29.99999698312937

In [7]:
function EncuentraTrancazosRaw(datos::Array, tolerancia=1400)
    result=Int[]
    longitud=length(datos)
    jcomp=0
    for j=1:longitud
        if abs(datos[j])>tolerancia
            if j-jcomp>1
                push!(result,j)
            end
            jcomp=j
        end
    end
    return result
end


EncuentraTrancazosRaw (generic function with 2 methods)

In [9]:
exemplo=vec(Datos["DatosCrudos"][2000,:]);

In [12]:
listongas=EncuentraTrancazosRaw(exemplo)

3-element Array{Int64,1}:
  43890
 114498
 185107

In [13]:
function ActivAlrededorTrancazo(Lista::Array, xxs::Array)
    #Aqui no se le ha hecho reshape a las matrices todavia
    result=Dict{AbstractString, Array}()
    q=1
    desde=20
    hasta=280
    for j in Lista
        nomineclave="Trancazo_$q"
        result[nomineclave]=xxs[:,j-desde:j+hasta]
        #println(nomineclave)
        q+=1
    end
    return result
end


ActivAlrededorTrancazo (generic function with 1 method)

In [14]:
ActividadRaw=ActivAlrededorTrancazo(listongas, Datos["DatosCrudos"])

Dict{AbstractString,Array{T,N}} with 3 entries:
  "Trancazo_2" => 4096x301 Array{UInt16,2}:…
  "Trancazo_1" => 4096x301 Array{UInt16,2}:…
  "Trancazo_3" => 4096x301 Array{UInt16,2}:…

In [16]:
function FormaMatrizDatosCentrados(xxs::Array)
    #El array tiene que ser de 4096 por algo mas
    irrrelevante,largo=size(xxs)
    aux=Array(Int32, 64,64, largo);
    for j=1:64,k=1:64
        aux[k,j,:]=xxs[j+(k-1)*64,:]
    end
    result=round(Int32,aux)*(-1)+2048;
    aux=0
    return result
end

FormaMatrizDatosCentrados (generic function with 1 method)

In [17]:
DatosCentrados=FormaMatrizDatosCentrados(ActividadRaw["Trancazo_1"])

64x64x301 Array{Int32,3}:
[:, :, 1] =
 2048   22    7    13   27   30   13  …   28   45   28   46   37    58   23
   47   35   57    39   31   64   37      16   23   13   22   36    53    9
   25   26   44     8   42    6   14      36   45   53   65   41    39   53
   15   20   17    18   18   -4   19      27   24   39   11   35    23   23
    5   16   10    25  -17   28    2      19   15    9   20   16  2048    6
   12   14   20     7   15   24   18  …    6    8   23    0    1     9    8
   12   23   11    19   15   25   16      23   32   30   25   34    37   25
   28   42   30    39   27   21   20      20   18    9   10   22    12   -1
   14   18   20    11   21   19   20      24   27   38   27   19    36   36
   46   31   27    31   34   31   32      20   11   19   14    8     6    2
    3   14   31    20   23   23   12  …   31   29   22   30   23    19   25
   12   13    7    26    8   16   12      24   28   27   17   34    17   24
   16   17   15    31   17   33   16      18   30 


Hay un montonal de canales que por la forma en que requiere ser hecho el experimento no detectaron nada o casi nada. Esos aparecen el registro como saturados en el máximo valor posible. Dado que esto es MUY inconveniente para el cálculo del CM, tenemos que detectarlos automáticamente y ponerlos en cero o ver si los esquivamos.